In [8]:
import pandas_gbq
import pandas as pd

In [ ]:
import requests
import os
import json

def shorten_url(long_url, api_token, alias=None, domain="tinyurl.com", tags=None):

    api_url = "https://api.tinyurl.com/create"

    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
        "Accept": "application/json", # Good practice to specify expected response type
    }

    payload = {
        "url": long_url,
        "domain": domain,
    }
    if alias:
        payload["alias"] = alias
    if tags:
        payload["tags"] = tags if isinstance(tags, list) else [tags] # Ensure tags is a list

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()

        result = response.json()

        if result.get("code") == 0 and result.get("data") and result["data"].get("tiny_url"):
            return result["data"]["tiny_url"]
        else:
            errors = result.get("errors", ["Unknown error from TinyURL API."])
            print(f"TinyURL API Error: {'; '.join(errors)}")
            print(f"Full response: {result}")
            return None

    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e.response.status_code}")
        try:
            # Try to parse error details from the response body if available
            error_details = e.response.json()
            print(f"Details: {error_details.get('errors', [e.response.text])}")
        except json.JSONDecodeError:
            print(f"Raw error response: {e.response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error connecting to TinyURL: {e}")
        return None
    except (json.JSONDecodeError, KeyError, TypeError) as e:
        print(f"Error parsing TinyURL response: {e}")
        print(f"Response text: {response.text if 'response' in locals() else 'No response object'}")
        return None

In [9]:
description= "La tabla estandar_aa constituye un detallado catálogo de estándares y buenas prácticas, diseñado para la evaluación y certificación en el sector agroindustrial. Cada entrada, identificada por un n numérico y un codigo alfanumérico único, se clasifica por su nivel de exigencia (como Fundamental o Avanzado) y se le asigna una puntos específica. Estos estándares se agrupan por dimension (Ambiente, Calidad, etc.) y tema (Agua, Biodiversidad), describiendo una buena_practica general, la accion concreta que la planta debe implementar, el medio_de_verificacion necesario para demostrar su cumplimiento, y un link_recursos para información adicional.",

In [10]:
table_schema = [
  {
    "name": "n",
    "type": "INTEGER",
    "mode": "REQUIRED",
    "description": "Identificador numérico único para cada fila, correspondiente al número de la fila en la planilla Excel de origen. Actúa como clave primaria de la fuente."
  },
  {
    "name": "codigo",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Código alfanumérico único que identifica cada estándar o requisito específico de buena práctica. Ejemplo: 'A001'."
  },
  {
    "name": "nivel",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Nivel de criticidad o avance de la buena práctica. Valores posibles incluyen: Fundamental, Básico, Intermedio, Avanzado."
  },
  {
    "name": "puntos",
    "type": "INTEGER",
    "mode": "NULLABLE",
    "description": "Puntaje asignado a la buena práctica o acción, posiblemente utilizado para evaluaciones de cumplimiento o certificación."
  },
  {
    "name": "dimension",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Categoría principal o pilar de sostenibilidad al que pertenece la buena práctica. Ejemplos: Ambiente, Calidad, Gestión, Social, Ética."
  },
  {
    "name": "tema",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Subcategoría o área temática específica dentro de una dimensión. Ejemplos: Agua (dentro de Ambiente), Gestión de la Calidad (dentro de Calidad)."
  },
  {
    "name": "buena_practica",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Enunciado general o título de la buena práctica que se busca implementar o cumplir bajo un tema y dimensión específicos."
  },
  {
    "name": "accion",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Descripción detallada de la tarea, actividad o medida concreta que la planta debe ejecutar para cumplir con la buena práctica."
  },
  {
    "name": "medio_de_verificacion",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Criterios, documentos, registros o inspecciones requeridas para demostrar y auditar el cumplimiento de la acción específica. Detalla qué evidencia se debe presentar."
  },
  {
    "name": "link_recursos",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Enlace web (URL) que dirige a recursos adicionales, normativas, guías o material de apoyo relevante para la buena práctica o acción."
  }
]

In [11]:
df_estandar = pd.read_excel("data/estandar_aa.xlsx")

In [ ]:
df_estandar["link_recursos"] = df_estandar.link_recursos.apply(lambda link_recursos: shorten_url(link_recursos, os.getenv("TINYURL_API_TOKEN")))

In [25]:
project_id = 'agro-extension-digital-npe'
destination_table = 'sandbox_rsolar.estandar_aa'

try:
    pandas_gbq.to_gbq(
        df_estandar,
        destination_table,
        project_id=project_id,
        if_exists='replace',
        table_schema=table_schema
    )
    print(f"DataFrame cargado exitosamente en {destination_table} con el esquema especificado.")

except Exception as e:
    print(f"Ocurrió un error: {e}")

100%|██████████| 1/1 [00:00<00:00, 4419.71it/s]

DataFrame cargado exitosamente en sandbox_rsolar.estandar_aa con el esquema especificado.


In [ ]:
table_schema = [
  {
    "name": "dimension",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Categoría principal o pilar temático al que se asocia el recurso. Define el área general de impacto o enfoque (ej. Ambiente, Calidad, Social, Ética, General, G. de la calidad)."
  },
  {
    "name": "tema",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Subcategoría o tópico específico dentro de la 'dimensión'. Proporciona un nivel de detalle mayor sobre el contenido del recurso (ej. Agua, Biodiversidad, Inocuidad, Condiciones de trabajo, Debida diligencia)."
  },
  {
    "name": "tipodetalle",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Clasificación del recurso según su formato o naturaleza. Indica qué tipo de material es (ej. Señalética, TDR - Términos de Referencia, Registro, Guía, Curso, Estándar)."
  },
  {
    "name": "detalle",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Título o nombre descriptivo completo del recurso. Es la denominación legible por humanos que identifica de manera única el contenido específico (ej. 'Evite el desperdicio de agua', 'TDR para la contratación de empresa que realice plan de gestión del recurso hídrico', 'Guía para la gestión sustentable del agua (adaptación predio y planta)')."
  },
  {
    "name": "codigo",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Identificador(es) alfanumérico(s) único(s) asignado(s) al recurso para su seguimiento y referencia interna, a menudo vinculados a normativas o puntos de control. Si existen múltiples códigos para un mismo recurso, se listan separados por comas y un espacio (ej. 'A007', 'A008, A009, A040, A059, A082')."
  },
  {
    "name": "link_web",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "URL principal que dirige a la página web o landing page del recurso, generalmente alojado en el dominio 'ciruelacertificada.cl'. Este enlace sirve como punto de acceso general al recurso."
  },
  {
    "name": "link_curso_chile_agricola",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "URL específica que dirige al material del curso en la plataforma 'chileagricola.cl', aplicable cuando el recurso es o está directamente asociado a un curso ofrecido en dicha plataforma."
  },
  {
    "name": "link_pdf",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "URL directa para acceder o descargar el recurso en formato PDF (Adobe Portable Document Format), si está disponible en esta presentación."
  },
  {
    "name": "link_word",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "URL directa para acceder o descargar el recurso en formato Microsoft Word (.doc o .docx), si está disponible en este formato editable de procesamiento de texto."
  },
  {
    "name": "link_excel",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "URL directa para acceder o descargar el recurso en formato Microsoft Excel (.xls o .xlsx), si está disponible en este formato de hoja de cálculo."
  }
]

In [ ]:
df_recursos = pd.read_excel("data/recursos_aa.xlsx")

In [ ]:
df_recursos['codigo_list'] = df_recursos['codigo'].str.split(r'\s*,\s*')

In [ ]:
df_unnested = df_recursos.explode('codigo_list')

In [ ]:
df_unnested = df_unnested.drop(columns=["codigo"]).reset_index(drop=True)

In [ ]:
df_unnested = df_unnested.rename(columns={'codigo_list': 'codigo'})
df_unnested['codigo'] = df_unnested['codigo'].str.strip()

In [ ]:
project_id = 'agro-extension-digital-npe'
destination_table = 'sandbox_rsolar.recursos_aa'

try:
    pandas_gbq.to_gbq(
        df_unnested,
        destination_table,
        project_id=project_id,
        if_exists='replace',
        table_schema=table_schema
    )
    print(f"DataFrame cargado exitosamente en {destination_table} con el esquema especificado.")

except Exception as e:
    print(f"Ocurrió un error: {e}")